In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.patches as patches
from time import process_time
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error as MAE
from sklearn.model_selection import GridSearchCV
from sklearn.tree import DecisionTreeRegressor
import psutil
import os
import re
import multiprocessing
import joblib
import gc
from tqdm.notebook import tqdm  # Progress bars for Jupyter

# Configuration options
VISUALIZE_RECTANGLES = False  # Set to True if you want to visualize rectangles
SAVE_INTERMEDIATE_MODELS = True  # Set to False to save only final models

# Flag to control whether to use multiple scales or only the maximum scale
use_multiple_scales = True  # Set to False to use only maximum scale, True for all scales

# Get available CPU cores and set appropriate parallelism
n_cores = multiprocessing.cpu_count()
n_jobs = max(1, n_cores - 1)  # Leave one core free for system processes
print(f"Using {n_jobs} of {n_cores} available CPU cores")

def monitor_memory():
    """Print current memory usage of the process"""
    process = psutil.Process()
    memory_mb = process.memory_info().rss / (1024 * 1024)
    print(f"Memory usage: {memory_mb:.2f} MB")

def MAPE(actual_values, predicted_values):
    """Calculate Mean Absolute Percentage Error with special handling for zeros"""
    # Vectorized implementation
    actual_flat = actual_values.flatten()
    pred_flat = predicted_values.flatten()
    
    # Create mask for non-zero actual values
    non_zero_mask = actual_flat != 0
    zero_mask = ~non_zero_mask
    
    # Calculate MAPE for non-zero elements
    mape_sum = 0
    count = len(actual_flat)
    
    if np.any(non_zero_mask):
        mape_sum += np.sum(np.abs((actual_flat[non_zero_mask] - pred_flat[non_zero_mask]) / actual_flat[non_zero_mask]))
    
    if np.any(zero_mask):
        mape_sum += np.sum(np.abs(actual_flat[zero_mask] - pred_flat[zero_mask]) / 100)
    
    return mape_sum / count

# Load spatial statistics to get universe boundaries for each dataset
print("Loading spatial statistics...")
spatial_stats = pd.read_csv('../spatial_statistics.csv')

# Directory containing the datasets
data_dir = '../large_files/resultsIntersects/'

# Parse bounding box information
def parse_bbox(bbox_str):
    # Extract coordinates from BOX string using regex
    pattern = r"BOX\(([-\d\.]+) ([-\d\.]+),([-\d\.]+) ([-\d\.]+)\)"
    match = re.search(pattern, bbox_str)
    if match:
        xmin = float(match.group(1))
        ymin = float(match.group(2))
        xmax = float(match.group(3))
        ymax = float(match.group(4))
        return xmin, ymin, xmax, ymax
    return -180, -90, 180, 90  # Default if parsing fails

# Extract universe boundaries for each dataset
universe_boundaries = {}
for _, row in spatial_stats.iterrows():
    table_name = row['Table Name']
    bbox = parse_bbox(row['Universe Limits (Bounding Box)'])
    universe_boundaries[table_name] = bbox

# Get list of all CSV files in the directory
print("Finding dataset files...")
csv_files = [f for f in os.listdir(data_dir) if f.endswith('.csv')]
print(f"Found {len(csv_files)} datasets to process")

# Define the scales of learning
scales = [1000, 5000, 10000, 50000, 100000, 500000, 1000000]

# Create necessary directories
os.makedirs('../large_files/LearnedModels/intersect/DT', exist_ok=True)
os.makedirs('../large_files/LearnedModels/intersect/DT/visualizations', exist_ok=True)
os.makedirs('../large_files/LearnedModels/intersect/DT/results', exist_ok=True)

# Lists to store all results
all_results_list = []

# Process each dataset
for csv_file in tqdm(csv_files, desc="Processing datasets"):
    # Force garbage collection at the start of each dataset
    gc.collect()
    monitor_memory()
    
    # Extract dataset name (remove "_results.csv")
    dataset_name = csv_file.replace('_results.csv', '')
    
    print(f"\nProcessing dataset: {dataset_name}")
    
    # Get universe boundaries for this dataset
    if dataset_name in universe_boundaries:
        univ_xmin, univ_ymin, univ_xmax, univ_ymax = universe_boundaries[dataset_name]
    else:
        # Default values if dataset not found in spatial stats
        univ_xmin, univ_ymin, univ_xmax, univ_ymax = -180, -90, 180, 90
    
    Surface_univ = (univ_xmax - univ_xmin) * (univ_ymax - univ_ymin)
    print(f"Universe boundaries for {dataset_name}: ({univ_xmin}, {univ_ymin}, {univ_xmax}, {univ_ymax})")
    
    # Load dataset - only load required columns
    data_path = os.path.join(data_dir, csv_file)
    print(f"Loading data from {data_path}")
    data = pd.read_csv(data_path, usecols=['Query MBR', 'Count MBR'])
    
    # Extract query MBR column (needs parsing as it's in string format)
    def parse_mbr(mbr_str):
        coords = mbr_str.strip('"()').split(', ')
        return [float(coord) for coord in coords]
    
    # Extract columns - use list comprehension for better performance
    print("Parsing MBR coordinates...")
    Rectangles = np.array([parse_mbr(mbr) for mbr in data['Query MBR']])
    Y = data[['Count MBR']].values  # Using Count MBR as target
    
    # Free up memory
    del data
    gc.collect()
    
    # Calculate basic statistics
    max_count = float(np.max(Y))
    min_count = float(np.min(Y))
    mean_count = float(np.mean(Y))
    median_count = float(np.median(Y))
    total_samples = len(Y)

    # Display basic statistics for the dataset
    print(f"\nBasic statistics for {dataset_name} dataset:")
    print(f"Max count: {max_count}")
    print(f"Min count: {min_count}")
    print(f"Mean count: {mean_count:.2f}")
    print(f"Median count: {median_count:.2f}")
    print(f"Total samples: {total_samples}\n")

    # Calculate rectangles density - vectorized version
    print("Calculating rectangle densities...")
    width = Rectangles[:, 2] - Rectangles[:, 0]
    height = Rectangles[:, 3] - Rectangles[:, 1]
    rectanglesDensity = np.abs(width * height / Surface_univ).reshape(-1, 1)
    
    # Prepare the dataset
    # X = np.append(Rectangles, rectanglesDensity, axis=1)
    X = Rectangles
    
    # Split the data into 80% train and 20% test
    print("Splitting data into train and test sets...")
    X_train, X_test_all, y_train, y_test_all = train_test_split(X, Y, test_size=0.2, random_state=3)
    
    # Visualize the first 1000 rectangles (only if enabled)
    if len(Rectangles) > 0 and VISUALIZE_RECTANGLES:
        print("Visualizing rectangles sample...")
        plt.figure(figsize=(10, 8))
        ax = plt.subplot()
        
        # Only visualize a sample to save time
        sample_size = min(1000, len(Rectangles))
        for i in range(sample_size):
            x1, y1, x2, y2 = Rectangles[i]
            color_val = float(rectanglesDensity[i][0]) if hasattr(rectanglesDensity[i], '__len__') else float(rectanglesDensity[i])
            rectangle = patches.Rectangle((x1, y1), x2 - x1, y2 - y1, 
                                        linewidth=1, edgecolor='b', facecolor='none', alpha=min(1.0, color_val*10))
            ax.add_patch(rectangle)
            
        plt.xlim(univ_xmin-20, univ_xmax+20)
        plt.ylim(univ_ymin-10, univ_ymax+10)
        plt.title(f"Sample rectangles from {dataset_name}")
        plt.savefig(f"../large_files/LearnedModels/intersect/DT/visualizations/{dataset_name}_rectangles.png", dpi=150)
        plt.close()  # Close to free memory instead of plt.show()
    
    # Adjust scales to the dataset size
    max_size = len(X_train)
    print(f"Training set size: {max_size}")

    if use_multiple_scales:
        # Use multiple scales as before
        adjusted_scales = [s for s in scales if s <= max_size]
        
        # Add intermediate 1 million increments for large datasets
        if max_size > 1000000:
            million_increments = list(range(2000000, max_size, 1000000))
            adjusted_scales.extend(million_increments)
            
        # Add the actual max size if it's not already in the list
        if max_size not in adjusted_scales:
            adjusted_scales.append(max_size)
            
        # Sort the scales to ensure they're in ascending order
        adjusted_scales.sort()
    else:
        # Use only the maximum scale
        adjusted_scales = [max_size]

    # List to store dataset-specific results
    dataset_results_list = []

    # Store best parameters from max scale training to reuse
    best_params = None
    
    # Process scales in reversed order (largest first)
    for sample_size in reversed(adjusted_scales):
        print(f"\nTraining with sample size: {sample_size}")
        monitor_memory()
        
        # Create training subset
        X_train_sample = X_train[:sample_size, :]
        y_train_sample = y_train[:sample_size]
        
        # Decision Tree Regressor parameters - optimized for performance
        params_dt = {
            "max_depth": [3, 4, 5, 6, 7, 8, 10],
            "min_samples_leaf": [0.04, 0.06, 0.08, 0.1, 0.13, 0.16, 0.2]
        }
        
        # Only do GridSearch for the max scale
        if sample_size == max_size or best_params is None:
            print("Performing grid search for optimal parameters...")
            dt = DecisionTreeRegressor(random_state=3)
            dt_cv = GridSearchCV(dt, params_dt, cv=5, n_jobs=n_jobs, verbose=1)
            
            # Time the grid search
            t1_start = process_time()
            dt_cv.fit(X_train_sample, y_train_sample)
            t1_stop = process_time()
            grid_search_time = t1_stop - t1_start
            
            # Store best parameters for reuse
            best_params = dt_cv.best_params_
            print(f"Grid search complete in {grid_search_time:.2f}s")
            print(f"Best parameters: {best_params}")
        else:
            # Skip grid search for smaller scales, use params from max scale
            dt_cv = None
            grid_search_time = 0
            print(f"Using best parameters from max scale: {best_params}")
        
        # Train the model with best parameters
        print("Training decision tree model...")
        dt = DecisionTreeRegressor(**best_params, random_state=3)
        t2_start = process_time()
        dt.fit(X_train_sample, y_train_sample)
        t2_stop = process_time()
        training_time = t2_stop - t2_start
        
        # Make predictions
        print("Making predictions...")
        y_pred = dt.predict(X_test_all)
        # Ensure predictions are non-negative (counts can't be negative)
        y_pred = np.maximum(0, y_pred)  # Convert any negative predictions to zero

        # Calculate metrics
        r2_score = dt.score(X_test_all, y_test_all)
        mae_value = MAE(y_test_all, y_pred)
        mape_value = MAPE(y_test_all, y_pred)
        
        # Calculate q-score - vectorized version
        print("Calculating performance metrics...")
        q_score_values = []
        
        # Vectorized q-score calculation
        y_true_flat = y_test_all.flatten()
        y_pred_flat = y_pred.flatten() if y_pred.ndim > 1 else y_pred
        
        # Find indices where both values are non-zero
        valid_indices = (y_true_flat != 0) & (y_pred_flat != 0)
        
        if np.any(valid_indices):
            ratios = np.maximum(
                y_pred_flat[valid_indices] / y_true_flat[valid_indices],
                y_true_flat[valid_indices] / y_pred_flat[valid_indices]
            )
            q_score_mean = np.mean(ratios)
        else:
            q_score_mean = 0
        
        # Time prediction performance (10 iterations)
        print("Measuring prediction performance...")
        total_duration = 0
        total_read = 0
        total_write = 0
        
        for _ in range(10):
            io_before = psutil.disk_io_counters()
            t3_start = process_time()
            preds = dt.predict(X_test_all)
            preds = np.maximum(0, preds)  # Include this operation in timing
            t3_stop = process_time()
            io_after = psutil.disk_io_counters()
            
            total_duration += (t3_stop - t3_start)
            total_read += io_after.read_count - io_before.read_count
            total_write += io_after.write_count - io_before.write_count
        
        avg_pred_time_microsec = (total_duration / 10) / len(y_pred) * 1000000
        avg_reads = total_read / 10 / len(y_pred)
        avg_writes = total_write / 10 / len(y_pred)
        
        # Save the model using joblib instead of pickle for better efficiency
        if SAVE_INTERMEDIATE_MODELS or sample_size == max_size:
            print("Saving model...")
            filename = f'../large_files/LearnedModels/intersect/DT/{dataset_name}_dt_{sample_size}_{training_time:.2f}s_{mape_value:.2%}_{mae_value:.2f}.joblib'
            joblib.dump(dt, filename, compress=3)
            # Get model file size in KB
            model_size_kb = os.path.getsize(filename) / 1024
            print(f"Model size: {model_size_kb:.2f} KB")
        else:
            model_size_kb = 0  # Set to 0 if model wasn't saved
        
        # Print results
        print(f"\nResults for {dataset_name}, Sample Size: {sample_size}")
        print(f"Grid Search Time: {grid_search_time:.2f}s, Training Time: {training_time:.2f}s")
        print(f"Decision Tree Parameters: {best_params}")
        print(f"Performance: R² = {r2_score:.4f}, MAE = {mae_value:.2f}, MAPE = {mape_value:.2%}")
        print(f"q-score: {q_score_mean:.2f}")
        print(f"Prediction time: {avg_pred_time_microsec:.4f} μs/sample")
        print(f"I/O: Reads={avg_reads:.6f}, Writes={avg_writes:.6f}")
        print("-" * 80)
        
        # Plot actual vs predicted only for the maximum scale
        if sample_size == adjusted_scales[-1]:  # Check if this is the maximum scale
            print("Generating prediction scatter plot...")
            plt.figure(figsize=(10, 8))
            plt.scatter(y_test_all, y_pred, s=0.5, alpha=0.5)
            plt.xlabel('True Values')
            plt.ylabel('Predictions')
            plt.title(f"{dataset_name} - Sample Size: {sample_size} (Maximum)")
            plt.grid(True, alpha=0.3)
            
            # Add diagonal line for perfect predictions
            max_val = max(np.max(y_test_all), np.max(y_pred))
            plt.plot([0, max_val], [0, max_val], 'r--', alpha=0.5)
            
            plt.savefig(f"../large_files/LearnedModels/intersect/DT/visualizations/{dataset_name}_{sample_size}_prediction.png", dpi=150)
            plt.close()  # Close to free memory
        
        # Store results in list (more efficient than DataFrame concat)
        result_row = {
            'Dataset': dataset_name,
            'Sample_Size': sample_size,
            'Training_Time': training_time,
            'Best_Params': str(best_params),
            'R2_Score': r2_score,
            'MAE': mae_value,
            'MAPE': float(mape_value),
            'Q_Score': q_score_mean,
            'Pred_Time_Microseconds': avg_pred_time_microsec,
            'IO_Reads': avg_reads,
            'IO_Writes': avg_writes,
            'Model_Size_KB': model_size_kb,
            'Max_Count': max_count,
            'Min_Count': min_count,
            'Mean_Count': mean_count,
            'Median_Count': median_count,
            'Total_Samples': total_samples
        }
        
        dataset_results_list.append(result_row)
        all_results_list.append(result_row)
        
        # Clean up to free memory
        if sample_size != max_size:  # Don't delete for max size as we might need it
            del X_train_sample, y_train_sample
            gc.collect()
    
    # Save results for this dataset
    print(f"Saving results for {dataset_name}...")
    dataset_results = pd.DataFrame(dataset_results_list)
    dataset_results.to_csv(f'../large_files/LearnedModels/intersect/DT/results/{dataset_name}_results.csv', index=False)
    
    # Clear memory before next dataset
    del X_train, X_test_all, y_train, y_test_all, Rectangles, Y, rectanglesDensity
    gc.collect()
    
# Save all results
print("Saving combined results...")
all_results = pd.DataFrame(all_results_list)
all_results.to_csv('../large_files/LearnedModels/intersect/DT/all_results.csv', index=False)

print("All processing completed and results saved.")
monitor_memory()

Using 29 of 30 available CPU cores
Loading spatial statistics...
Finding dataset files...
Found 14 datasets to process


Processing datasets:   0%|          | 0/14 [00:00<?, ?it/s]

Memory usage: 199.48 MB

Processing dataset: historicthingwaysorted
Universe boundaries for historicthingwaysorted: (-179.99526020000002, -85.0036942, 179.99597930000002, 78.06750650000001)
Loading data from ../large_files/resultsIntersects/historicthingwaysorted_results.csv


Parsing MBR coordinates...



Basic statistics for historicthingwaysorted dataset:
Max count: 1792176.0
Min count: 0.0
Mean count: 29765.11
Median count: 16.00
Total samples: 358439

Calculating rectangle densities...
Splitting data into train and test sets...
Training set size: 286751

Training with sample size: 286751
Memory usage: 245.47 MB
Performing grid search for optimal parameters...
Fitting 5 folds for each of 49 candidates, totalling 245 fits


...

Saving results for emergencythingwaysorted...
Memory usage: 540.48 MB

Processing dataset: craftwaysorted
Universe boundaries for craftwaysorted: (-175.2000514, -65.2458821, 175.3397782, 69.6673353)
Loading data from ../large_files/resultsIntersects/craftwaysorted_results.csv
Parsing MBR coordinates...



Basic statistics for craftwaysorted dataset:
Max count: 108929.0
Min count: 0.0
Mean count: 1705.24
Median count: 0.00
Total samples: 21822

Calculating rectangle densities...
Splitting data into train and test sets...
Training set size: 17457

Training with sample size: 17457
Memory usage: 540.48 MB
Performing grid search for optimal parameters...
Fitting 5 folds for each of 49 candidates, totalling 245 fits


Grid search complete in 0.38s
Best parameters: {'max_depth': 8, 'min_samples_leaf': 0.04}
Training decision tree model...
Making predictions...
Calculating performance metrics...
Measuring prediction performance...
Saving model...
Model size: 1.95 KB

Results for craftwaysorted, Sample Size: 17457
Grid Search Time: 0.38s, Training Time: 0.05s
Decision Tree Parameters: {'max_depth': 8, 'min_samples_leaf': 0.04}
Performance: R² = 0.2897, MAE = 2216.30, MAPE = 1633.44%
q-score: 32.21
Prediction time: 0.0877 μs/sample
I/O: Reads=0.000000, Writes=0.000000
--------------------------------------------------------------------------------
Generating prediction scatter plot...



Training with sample size: 10000
Memory usage: 540.48 MB
Using best parameters from max scale: {'max_depth': 8, 'min_samples_leaf': 0.04}
Training decision tree model...
Making predictions...
Calculating performance metrics...
Measuring prediction performance...
Saving model...
Model size: 1.88 KB

Results for craftwaysorted, Sample Size: 10000
Grid Search Time: 0.00s, Training Time: 0.03s
Decision Tree Parameters: {'max_depth': 8, 'min_samples_leaf': 0.04}
Performance: R² = 0.3018, MAE = 2202.65, MAPE = 1616.84%
q-score: 31.72
Prediction time: 0.0871 μs/sample
I/O: Reads=0.000000, Writes=0.000000
--------------------------------------------------------------------------------

Training with sample size: 5000
Memory usage: 540.48 MB
Using best parameters from max scale: {'max_depth': 8, 'min_samples_leaf': 0.04}
Training decision tree model...
Making predictions...
Calculating performance metrics...
Measuring prediction performance...
Saving model...
Model size: 1.85 KB

Results for c


Training with sample size: 1000
Memory usage: 540.48 MB
Using best parameters from max scale: {'max_depth': 8, 'min_samples_leaf': 0.04}
Training decision tree model...
Making predictions...
Calculating performance metrics...
Measuring prediction performance...
Saving model...
Model size: 1.78 KB

Results for craftwaysorted, Sample Size: 1000
Grid Search Time: 0.00s, Training Time: 0.00s
Decision Tree Parameters: {'max_depth': 8, 'min_samples_leaf': 0.04}
Performance: R² = 0.2376, MAE = 2290.30, MAPE = 1634.35%
q-score: 33.06
Prediction time: 0.0856 μs/sample
I/O: Reads=0.000000, Writes=0.000000
--------------------------------------------------------------------------------
Saving results for craftwaysorted...
Saving combined results...
All processing completed and results saved.
Memory usage: 540.48 MB
